In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

if not tf.__version__.startswith('2'):
    raise ValueError('This code requires TensorFlow V2.x')

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\rsshi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVDRQQA\imdb_reviews-train…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\rsshi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVDRQQA\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\rsshi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVDRQQA\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\rsshi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [3]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)

# Tokenize and pad the reviews
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
train_sentences, train_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in train_data])
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

test_sentences, test_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in test_data])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [4]:
##MODEL 1

In [5]:
model = keras.models.Sequential([
    keras.layers.Embedding(10000, 64),
    keras.layers.SimpleRNN(64),
    keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
782/782 [==============================] - 41s 49ms/step - loss: 0.6924 - accuracy: 0.5217 - val_loss: 0.6975 - val_accuracy: 0.5062
Epoch 2/10
782/782 [==============================] - 38s 49ms/step - loss: 0.6511 - accuracy: 0.6178 - val_loss: 0.6977 - val_accuracy: 0.5324
Epoch 3/10
782/782 [==============================] - 38s 49ms/step - loss: 0.4966 - accuracy: 0.7596 - val_loss: 0.7377 - val_accuracy: 0.5784
Epoch 4/10
782/782 [==============================] - 37s 48ms/step - loss: 0.3695 - accuracy: 0.8299 - val_loss: 0.9127 - val_accuracy: 0.5508
Epoch 5/10
782/782 [==============================] - 36s 46ms/step - loss: 0.2471 - accuracy: 0.8952 - val_loss: 1.1579 - val_accuracy: 0.5109
Epoch 6/10
782/782 [==============================] - 38s 48ms/step - loss: 0.2218 - accuracy: 0.9019 - val_loss: 1.1100 - val_accuracy: 0.5210
Epoch 7/10
782/782 [==============================] - 37s 47ms/step - loss: 0.1725 - accuracy: 0.9222 - val_loss: 1.4632 - val_accuracy:

In [8]:
##MODEL 2 - RNN with Improvements 

In [9]:
model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32), # Reduced embedding dimension
    keras.layers.Dropout(0.5), # Added dropout
    keras.layers.SimpleRNN(32, kernel_regularizer=keras.regularizers.l2(0.001)), # L2 regularization
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Adding early stopping callback
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop])

Epoch 1/10
782/782 [==============================] - 42s 51ms/step - loss: 0.7155 - accuracy: 0.5222 - val_loss: 0.7028 - val_accuracy: 0.5506
Epoch 2/10
782/782 [==============================] - 39s 49ms/step - loss: 0.6847 - accuracy: 0.5922 - val_loss: 0.5953 - val_accuracy: 0.7187
Epoch 3/10
782/782 [==============================] - 38s 49ms/step - loss: 0.5735 - accuracy: 0.7211 - val_loss: 0.6888 - val_accuracy: 0.5968
Epoch 4/10
782/782 [==============================] - 38s 49ms/step - loss: 0.5611 - accuracy: 0.7216 - val_loss: 0.7241 - val_accuracy: 0.5603
Epoch 5/10
782/782 [==============================] - 38s 48ms/step - loss: 0.4288 - accuracy: 0.8188 - val_loss: 0.5826 - val_accuracy: 0.7668
Epoch 6/10
782/782 [==============================] - 38s 48ms/step - loss: 0.3252 - accuracy: 0.8741 - val_loss: 0.6441 - val_accuracy: 0.7350
Epoch 7/10
782/782 [==============================] - 38s 48ms/step - loss: 0.2612 - accuracy: 0.9043 - val_loss: 0.6268 - val_accuracy:

In [10]:
keras.layers.Embedding(10000, 32) # Reduced from 64 to 32 

In [11]:
keras.layers.Dropout(0.5) # 50% dropout rate 

In [12]:
keras.layers.SimpleRNN(32, kernel_regularizer=keras.regularizers.l2(0.001)) #regularization on rnn layer L2

In [13]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
##MODEL 3 - LSTM

In [14]:
from keras.layers import Bidirectional, LSTM

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop])

Epoch 1/10
782/782 [==============================] - 134s 157ms/step - loss: 0.5257 - accuracy: 0.7311 - val_loss: 0.4254 - val_accuracy: 0.8096
Epoch 2/10
782/782 [==============================] - 121s 155ms/step - loss: 0.3464 - accuracy: 0.8628 - val_loss: 0.4365 - val_accuracy: 0.8151
Epoch 3/10
782/782 [==============================] - 122s 156ms/step - loss: 0.2962 - accuracy: 0.8858 - val_loss: 0.5029 - val_accuracy: 0.7993
Epoch 4/10
782/782 [==============================] - 122s 156ms/step - loss: 0.2671 - accuracy: 0.9003 - val_loss: 0.4984 - val_accuracy: 0.7865


In [ ]:
##MODEL 4 - CNN 

In [15]:
from keras.layers import Conv1D, MaxPooling1D

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Conv1D(32, 7, activation='relu'),
    MaxPooling1D(5),
    Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop, checkpoint])

Epoch 1/10
782/782 [==============================] - 84s 81ms/step - loss: 0.5797 - accuracy: 0.6935 - val_loss: 0.4524 - val_accuracy: 0.7874
Epoch 2/10
782/782 [==============================] - 59s 75ms/step - loss: 0.3683 - accuracy: 0.8416 - val_loss: 0.4262 - val_accuracy: 0.8029
Epoch 3/10
782/782 [==============================] - 59s 76ms/step - loss: 0.3125 - accuracy: 0.8762 - val_loss: 0.4313 - val_accuracy: 0.8107
Epoch 4/10
782/782 [==============================] - 59s 75ms/step - loss: 0.2768 - accuracy: 0.8925 - val_loss: 0.4493 - val_accuracy: 0.7958
Epoch 5/10
782/782 [==============================] - 58s 74ms/step - loss: 0.2412 - accuracy: 0.9103 - val_loss: 0.4926 - val_accuracy: 0.7992
